In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from shapely.wkt import dumps

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

## Downloading network

In [2]:
USOS_ZONA_7 = gpd.read_file("USOS_ZONA_7.shp")

In [3]:
print(USOS_ZONA_7.crs)

EPSG:32613


In [3]:
USOS_ZONA_7 = USOS_ZONA_7.to_crs("EPSG:4326")

In [4]:
# Convertir la geometría del polígono en WKT (para usar en la query)
poly_wkt = dumps(USOS_ZONA_7.dissolve().geometry.iloc[0])

In [5]:
schema = "denue"
table = "denue_2022"

manzana = "manzana"
entidad = "cve_ent"
localidad = "cve_loc"
municipio = "cve_mun"
id = "id"
latitud = "latitud"
longitud = "longitud"
codigo_act = "codigo_act"
ageb = "ageb"
per_ocu = "per_ocu"

query_censo = f"""
SELECT 
  "{id}", 
  "{entidad}",
  "{localidad}",
  "{manzana}",
  "{municipio}",
  "{latitud}",
  "{longitud}",
  "{codigo_act}",
  "{ageb}",
  "{per_ocu}",
  "geometry"
FROM {schema}.{table}
WHERE ST_Intersects(geometry, ST_GeomFromText('{poly_wkt}', 4326))"""

# Ejecutar la consulta y cargar los datos en un GeoDataFrame
Denue = aup.gdf_from_query(query_censo, geometry_col='geometry')

print(Denue.shape)
Denue.head()

(1910, 11)


,id,cve_ent,cve_loc,manzana,cve_mun,latitud,longitud,codigo_act,ageb,per_ocu,geometry
0,6897119,14,0001,010,120,20.687820,-103.455900,522110,4836,0 a 5 personas,POINT (-103.45590 20.68782)
1,6825551,14,0001,010,120,20.687842,-103.455893,522110,4836,0 a 5 personas,POINT (-103.45589 20.68784)
2,8650318,14,0001,008,120,20.631353,-103.443319,315223,2384,11 a 30 personas,POINT (-103.44332 20.63135)
3,1822560,14,0001,008,120,20.631702,-103.444105,461110,2384,0 a 5 personas,POINT (-103.44411 20.63170)
4,1822559,14,0001,008,120,20.631654,-103.444090,461121,2384,0 a 5 personas,POINT (-103.44409 20.63165)


## Clasificar las distintas actividades económicas con base en su terminación en "codigo_act" basándonos en el Directorio Estadístico Nacional de Unidades Económicas

In [6]:
# Función para clasificar según la terminación de codigo_act
def asignar_tipo(codigo):
    if pd.isna(codigo):  # Si está vacío
        return 'Sin código'
    
    # Asegurar que sea string para evaluar el código
    codigo_str = str(codigo).strip()

    if not codigo_str.isdigit():
        return 'Código inválido'
    
    # Define tus conjuntos de códigos
    industria = {'11', '21', '23', '31', '32', '33', '55'}
    
    servicios = {'22', '48', '49', '52', '53', '54', '56', '72',
                 '81'}
    
    comercio = {'43', '46'}
    
    cultural_recreativo = {'51','71'}
    
    educacion = {'61'}
    
    salud = {'62'}

    gobierno = {'93'}
    
    # Verificar si el código está en alguno de los conjuntos
    if codigo_str[:2] in industria:
        return 'Industria'
    elif codigo_str[:2] in servicios:
        return 'Servicios'
    elif codigo_str[:2] in comercio:
        return 'Comercio'
    elif codigo_str[:2] in cultural_recreativo:
        return 'Cultural/Recreativo'
    elif codigo_str[:2] in educacion:
        return 'Educación'
    elif codigo_str[:2] in salud:
        return 'Salud'
    elif codigo_str[:2] in gobierno:
        return 'Gobierno'
    else:
        return 'Desconocido'

# Aplica la función al DataFrame
Denue['tipo_act'] = Denue['codigo_act'].apply(asignar_tipo)

In [36]:
Denue.head(3)

,id,cve_ent,cve_loc,manzana,cve_mun,latitud,longitud,codigo_act,ageb,per_ocu,geometry,tipo_act
0,6897119,14,0001,010,120,20.687820,-103.455900,522110,4836,0 a 5 personas,POINT (-103.45590 20.68782),Servicios
1,6825551,14,0001,010,120,20.687842,-103.455893,522110,4836,0 a 5 personas,POINT (-103.45589 20.68784),Servicios
2,8650318,14,0001,008,120,20.631353,-103.443319,315223,2384,11 a 30 personas,POINT (-103.44332 20.63135),Industria


In [37]:
Denue["tipo_act"].unique()

array(['Servicios', 'Industria', 'Comercio', 'Educación', 'Salud',
       'Cultural/Recreativo', 'Gobierno'], dtype=object)

In [38]:
desconocidos = Denue[Denue['tipo_act'] == 'Desconocido']

# Mostrar los valores únicos de codigo_act que no clasificaron
desconocidos_codigos = desconocidos['codigo_act'].unique()

print("Códigos que quedaron como 'Desconocido':")
print(desconocidos_codigos)

Códigos que quedaron como 'Desconocido':
[]


In [39]:
# Contar cuántas veces aparece cada código no reconocido
desconocidos['codigo_act'].value_counts()

Series([], Name: count, dtype: int64)

In [40]:
# Total de registros marcados como 'Desconocido'
total_desconocidos = (Denue['tipo_act'] == 'Desconocido').sum()

print(f"Total de 'Desconocido': {total_desconocidos}")

Total de 'Desconocido': 0


### Relación espacio -> espacio

#### Mostrar de manera completa las relaciones y jerarquías entre los distintos espacios

In [7]:
jerarquia_completa = Denue[['cve_ent', 'cve_mun', 'cve_loc', 'ageb', 'manzana']].drop_duplicates().sort_values(by=['cve_ent', 'cve_mun', 'cve_loc', 'ageb', 'manzana'])

jerarquia_completa.head(10)

,cve_ent,cve_mun,cve_loc,ageb,manzana
112,14,120,0001,1687,001
1817,14,120,0001,1687,002
609,14,120,0001,1687,003
113,14,120,0001,1687,004
1874,14,120,0001,1687,005
1815,14,120,0001,1687,007
316,14,120,0001,1687,009
592,14,120,0001,1687,011
111,14,120,0001,1687,012
179,14,120,0001,1687,013


In [8]:
conteo_manzanas = jerarquia_completa.groupby(['cve_ent', 'cve_mun', 'cve_loc', 'ageb', 'manzana']) \
    .size().reset_index(name='cont_manzanas') #  Contar cuántas manzanas hay por agrupación y convertimos a string

In [9]:
conteo_manzanas.head(5)

,cve_ent,cve_mun,cve_loc,ageb,manzana,cont_manzanas
0,14,120,0001,1687,001,1
1,14,120,0001,1687,002,1
2,14,120,0001,1687,003,1
3,14,120,0001,1687,004,1
4,14,120,0001,1687,005,1


In [10]:
conteo_manzanas["cont_manzanas"].unique()

array([1])

In [11]:
# Extraer los primeros N dígitos de cada variable
jerarquia_completa['entidad_str'] = jerarquia_completa['cve_ent'].str[:2]  # 2 primeros dígitos de entidad
jerarquia_completa['municipio_str'] = jerarquia_completa['cve_mun'].str[:3]  # 3 primeros dígitos de municipio
jerarquia_completa['localidad_str'] = jerarquia_completa['cve_loc'].str[:4]  # 4 primeros dígitos de localidad
jerarquia_completa['ageb_str'] = jerarquia_completa['ageb'].str[:4]  # 4 primeros dígitos de AGEB
jerarquia_completa['manzana_str'] = jerarquia_completa['manzana'].str[:3]  # 3 primeros dígitos de manzana

In [12]:
jerarquia_completa['cod_16d'] = (
    jerarquia_completa['entidad_str'] + '-' +
    jerarquia_completa['municipio_str'] + '-' +
    jerarquia_completa['localidad_str'] + '-' +
    jerarquia_completa['ageb_str'] + '-' +
    jerarquia_completa['manzana_str']
)

In [13]:
jerarquia_completa[['cve_ent', 'entidad_str', 'cve_mun', 'municipio_str', 'cod_16d']].head()

,cve_ent,entidad_str,cve_mun,municipio_str,cod_16d
112,14,14,120,120,14-120-0001-1687-001
1817,14,14,120,120,14-120-0001-1687-002
609,14,14,120,120,14-120-0001-1687-003
113,14,14,120,120,14-120-0001-1687-004
1874,14,14,120,120,14-120-0001-1687-005


In [15]:
Denue_completo = Denue.merge(
    jerarquia_completa)

Denue_completo.head(3)

,id,cve_ent,cve_loc,manzana,cve_mun,latitud,longitud,codigo_act,ageb,per_ocu,geometry,tipo_act,entidad_str,municipio_str,localidad_str,ageb_str,manzana_str,cod_16d
0,6897119,14,0001,010,120,20.687820,-103.455900,522110,4836,0 a 5 personas,POINT (-103.45590 20.68782),Servicios,14,120,0001,4836,010,14-120-0001-4836-010
1,6825551,14,0001,010,120,20.687842,-103.455893,522110,4836,0 a 5 personas,POINT (-103.45589 20.68784),Servicios,14,120,0001,4836,010,14-120-0001-4836-010
2,7304814,14,0001,010,120,20.674227,-103.457050,434211,4836,0 a 5 personas,POINT (-103.45705 20.67423),Comercio,14,120,0001,4836,010,14-120-0001-4836-010


## Importar las manzanas de censo

In [16]:
schema = "sociodemografico"
table = "censo_inegi_20_mza"

entidad = "cve_ent"
localidad = "cve_loc"
municipio = "cve_mun"
manzana = "cve_mza"
ageb = "cve_ageb"
ambito = "ambito"
tipomza = "tipomza"

query_censo = f"""
SELECT 
"{entidad}",
"{manzana}",
"{localidad}",
"{municipio}",
"{ageb}",
"{ambito}",
"{tipomza}",
"geometry"
FROM {schema}.{table} WHERE ST_Intersects(geometry, ST_GeomFromText('{poly_wkt}', 4326))"""

# Ejecutar la consulta y cargar los datos en un GeoDataFrame
censo_denue = aup.gdf_from_query(query_censo, geometry_col='geometry')

print(censo_denue.shape)
censo_denue.head()

(1588, 8)


,cve_ent,cve_mza,cve_loc,cve_mun,cve_ageb,ambito,tipomza,geometry
0,14,002,0001,120,4840,Urbana,Típica,"POLYGON ((-103.45411 20.66596, -103.45461 20.6..."
1,14,004,0001,120,1687,Urbana,Típica,"POLYGON ((-103.45328 20.64672, -103.45334 20.6..."
2,14,005,0001,120,1687,Urbana,Típica,"POLYGON ((-103.45381 20.64695, -103.45525 20.6..."
3,14,008,0001,120,1687,Urbana,Típica,"POLYGON ((-103.45294 20.64779, -103.45364 20.6..."
4,14,017,0001,120,1687,Urbana,Típica,"POLYGON ((-103.45382 20.64688, -103.45386 20.6..."


### Contabilizar cuántas personas hay ejerciendo cada actividad económica

In [17]:
def number_of_jobs(per_ocu):
    jobs_dict = {'0 a 5 personas':3,
                '6 a 10 personas':8,
                '11 a 30 personas':20,
                '31 a 50 personas':40,
                '51 a 100 personas':75,
                '101 a 250 personas':175,
                '251 y más personas':325}
    per_ocu_num = jobs_dict[per_ocu]
    return per_ocu_num

In [18]:
Denue_completo['per_ocu_num'] = Denue_completo.per_ocu.apply(lambda per_ocu: number_of_jobs(per_ocu))
Denue_completo.head(2)

,id,cve_ent,cve_loc,manzana,cve_mun,latitud,longitud,codigo_act,ageb,per_ocu,geometry,tipo_act,entidad_str,municipio_str,localidad_str,ageb_str,manzana_str,cod_16d,per_ocu_num
0,6897119,14,0001,010,120,20.687820,-103.455900,522110,4836,0 a 5 personas,POINT (-103.45590 20.68782),Servicios,14,120,0001,4836,010,14-120-0001-4836-010,3
1,6825551,14,0001,010,120,20.687842,-103.455893,522110,4836,0 a 5 personas,POINT (-103.45589 20.68784),Servicios,14,120,0001,4836,010,14-120-0001-4836-010,3


### Hacer el match entre las manzanas del denue y las manzanas del censo

In [19]:
censo_denue.rename(columns = {"cve_ageb": "ageb"} , inplace = True)
censo_denue.rename(columns = {"cve_mza": "manzana"} , inplace = True)

In [42]:
# Filtrar solo las columnas relevantes de Denue_completo
Denue_completo_filtrado = Denue_completo[['cve_ent', 'cve_mun', 'cve_loc', 'ageb', 'manzana', 
                                          'codigo_act', 'per_ocu', 'cod_16d', 'per_ocu_num']]

# Hacer el merge para agregar las columnas deseadas
censo_denue_filtrado = Denue_completo[Denue_completo[['cve_ent', 'cve_mun', 'cve_loc', 'ageb', 'manzana']]
                      .apply(tuple, axis=1)
                      .isin(censo_denue[['cve_ent', 'cve_mun', 'cve_loc', 'ageb', 'manzana']]
                      .apply(tuple, axis=1))]

# Hacer el merge para agregar las columnas pero manteniendo solo la geometría de censo_denue
censo_denue_filtrado = censo_denue_filtrado[['cve_ent', 'cve_mun', 'cve_loc', 'ageb', 'manzana', 'geometry']].merge(
    Denue_completo_filtrado, 
    on=['cve_ent', 'cve_mun', 'cve_loc', 'ageb', 'manzana'], 
    how='left'
)

# Verificar el resultado
print(censo_denue_filtrado.shape)
censo_denue_filtrado.head(3)

KeyError: "['geometry'] not in index"

In [41]:
censo_denue_filtrado.columns.tolist()

['id',
 'cve_ent',
 'cve_loc',
 'manzana',
 'cve_mun',
 'latitud',
 'longitud',
 'codigo_act_x',
 'ageb',
 'per_ocu_x',
 'tipo_act',
 'entidad_str',
 'municipio_str',
 'localidad_str',
 'ageb_str',
 'manzana_str',
 'cod_16d_x',
 'per_ocu_num_x',
 'codigo_act_y',
 'per_ocu_y',
 'cod_16d_y',
 'per_ocu_num_y']

In [34]:
censo_denue_filtrado.drop(columns = ["per_ocu_y", "cod_16d_y", "per_ocu_num_y", "per_ocu_num_y"], inplace = True)

In [35]:
censo_denue_filtrado.drop(columns = "codigo_act_y", inplace = True)

In [36]:
censo_denue_filtrado.rename(columns = {"per_ocu_x": "per_ocu", "cod_16d_x": "cod_16d", "codigo_act_x": "codigo_act", "per_ocu_num_x": "per_ocu_num"}, 
                           inplace = True)

In [37]:
censo_denue_filtrado.columns.tolist()

['id',
 'cve_ent',
 'cve_loc',
 'manzana',
 'cve_mun',
 'latitud',
 'longitud',
 'codigo_act',
 'ageb',
 'per_ocu',
 'geometry',
 'tipo_act',
 'entidad_str',
 'municipio_str',
 'localidad_str',
 'ageb_str',
 'manzana_str',
 'cod_16d',
 'per_ocu_num']

In [38]:
censo_denue_filtrado.head(3)

,id,cve_ent,cve_loc,manzana,cve_mun,latitud,longitud,codigo_act,ageb,per_ocu,geometry,tipo_act,entidad_str,municipio_str,localidad_str,ageb_str,manzana_str,cod_16d,per_ocu_num
0,6897119,14,0001,010,120,20.68782,-103.4559,522110,4836,0 a 5 personas,POINT (-103.45590 20.68782),Servicios,14,120,0001,4836,010,14-120-0001-4836-010,3
1,6897119,14,0001,010,120,20.68782,-103.4559,522110,4836,0 a 5 personas,POINT (-103.45590 20.68782),Servicios,14,120,0001,4836,010,14-120-0001-4836-010,3
2,6897119,14,0001,010,120,20.68782,-103.4559,522110,4836,0 a 5 personas,POINT (-103.45590 20.68782),Servicios,14,120,0001,4836,010,14-120-0001-4836-010,3


## Crear centroides

In [9]:
print(Denue.crs)

epsg:4326


In [ ]:
jerarquia_completa = jerarquia_completa.merge(
    Denue[['manzana', 'geometry', 'per_ocu']],
    on='manzana', 
    how='left'
)

# Convertir en GeoDataFrame
jerarquia_completa = gpd.GeoDataFrame(jerarquia_completa, geometry='geometry')

jerarquia_completa.set_crs(epsg=4326, inplace=True)

In [12]:
# Reproyectar
jerarquia_completa.set_crs = 32614

# Calcular el centroide
jerarquia_completa['centroide'] = jerarquia_completa['geometry'].centroid

# Generar un buffer de 500 metros
jerarquia_completa['buffer_500m'] = jerarquia_completa['centroide'].buffer(500)

# Visualizar en el sistema original EPSG:4326 para mapas web
jerarquia_completa_buffers = jerarquia_completa.set_geometry('buffer_500m')
jerarquia_completa_buffers = jerarquia_completa_buffers.to_crs(epsg=4326)

KeyError: 'geometry'

In [ ]:
# Visualiza en geopandas
jerarquia_completa_buffers.plot()